In [1]:
import pandas as pd

/var/folders/4c/ydvq3h253pj9vrlq1pxpv9l40000gn/T/ipykernel_6979/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
amazon_data = pd.read_csv('../amazon_best_sellers.csv')

In [5]:
amazon_data.columns

Index(['product url', 'ranking', 'brand', 'product name', 'number of ratings',
       'star rating', 'price(in dollar)', 'color', 'compatible devices',
       'connectivity technology', 'connector type', 'date first available'],
      dtype='object')

In [6]:
amazon_cleaned = amazon_data[amazon_data.columns]
# 1. Converting 'number of ratings' to numerical format
amazon_cleaned['number of ratings'] = (
    amazon_cleaned['number of ratings']
    .str.replace(',', '')
    .str.extract(r'(\d+)')[0]
    .astype(float)
)

# 2. Converting 'price(in dollar)' to numerical format
amazon_cleaned['price(in dollar)'] = (
    amazon_cleaned['price(in dollar)']
    .str.extract(r'(\d+\.\d+|\d+)')[0]
    .astype(float)
)

# 3. Handling missing data: Replacing specific placeholder text with NaN
placeholders = ['data not available', 'Color not available', 'not available']
for placeholder in placeholders:
    amazon_cleaned = amazon_cleaned.replace(placeholder, pd.NA)

# 4. Standardizing text data: Lowercasing text columns
text_columns = amazon_cleaned.select_dtypes(include=['object']).columns
amazon_cleaned[text_columns] = amazon_cleaned[text_columns].apply(lambda x: x.str.lower())

amazon_cleaned.head()

,product url,ranking,brand,product name,number of ratings,star rating,price(in dollar),color,compatible devices,connectivity technology,connector type,date first available
0,https://www.amazon.com/all-new-fire-hd-8-table...,#1,amazon,"amazon fire hd 8 tablet, 8” hd display, 32 gb,...",12280.0,4.5,64.99,<NA>,compatible devices data not available,connectivity technology data not available,connector type data not available,date first available data not available
1,https://www.amazon.com/mac-book-pro-charger-ca...,#2,sunveza,mac book pro charger - 118w usb c charger fast...,2926.0,4.6,NaN,118w macbook pro charger-white,NaN,usb,usb type c,"may 17, 2022"
2,https://www.amazon.com/amazon-fire-kids-tablet...,#3,amazon,"amazon fire 7 kids tablet, ages 3-7. top-selli...",14431.0,4.5,NaN,<NA>,compatible devices data not available,connectivity technology data not available,connector type data not available,date first available data not available
3,https://www.amazon.com/charger-computer-pavili...,#4,hibbo,charger for hp laptop computer 65w 45w smart b...,2389.0,4.7,NaN,black,NaN,connectivity technology data not available,4.5 x 3.0mm blue tip,"june 8, 2020"
4,https://www.amazon.com/wifi-6-router-gigabit-w...,#5,tp-link,tp-link ax1800 wifi 6 router (archer ax21) – d...,17268.0,4.3,NaN,<NA>,laptop,"wi-fi, ethernet",connector type data not available,"august 28, 2020"


In [7]:
amazon_cleaned['connectivity technology'].unique()

array(['connectivity technology data not available', 'usb',
       'wi-fi, ethernet', 't tip', 't-tip', 'wired', 'ethernet',
       'powerline', 'charger', 'wi-fi', 'wi-fi, usb, ethernet',
       'connector size: 4.5mm tip', 'magnetic'], dtype=object)

In [8]:
amazon_cleaned['date first available'].unique()

array(['date first available data not available', ' may 17, 2022 ',
       ' june 8, 2020 ', ' august 28, 2020 ', ' november 19, 2020 ',
       ' january 31, 2022 ', ' october 15, 2019 ',
       '\n                \u200ejune 22, 2022 ',
       '\n                \u200eseptember 16, 2021 ',
       ' november 10, 2023 ',
       '\n                \u200enovember 30, 2021 ',
       '\n                \u200edecember 3, 2020 ', ' july 19, 2022 ',
       ' july 10, 2019 ', ' september 23, 2019 ',
       '\n                \u200eseptember 24, 2020 ', ' january 1, 2017 ',
       '\n                \u200eseptember 30, 2014 ',
       '\n                \u200efebruary 22, 2021 ', ' october 4, 2020 ',
       ' july 3, 2021 ', ' june 26, 2023 ',
       '\n                \u200eaugust 4, 2021 ',
       '\n                \u200emay 31, 2021 ', ' september 16, 2021 ',
       ' april 5, 2021 ', ' may 19, 2022 ',
       '\n                \u200edecember 22, 2020 ',
       '\n                \u200ejanuary

In [9]:
# Checking the 'color' column for non-color values and cleaning it
unique_colors = amazon_cleaned['color'].unique()
unique_colors

array([<NA>, '118w macbook pro charger-white', 'black', 'white',
       'space grey', 'space gray', '45w', 'silver',
       'macbook pro charger-white', 'quote stickers', 'grey',
       'black+blue', 'colorful', '\u200ewhite-96w', 'usb-c', 'pa-45w',
       'blue', 'dale pink', '1', 'black 45w', '106', 'dell 45w',
       'tq50-0092 kc football', 'opal', '200 pcs', 'snowflake white',
       'multicolor', 'animal stickers', 'midnight', 'gray+pd', 'gray'],
      dtype=object)

In [10]:
# Cleaning strategy: Replace non-color values with NaN
common_colors = ['multicolor', 'black', 'silver', 'white', 'blue', 'pink', 'grey']

# Function to clean the color column
def clean_color(value):
    if pd.isna(value):
        return value
    for color in common_colors:
        if color in value:
            return color
    if 'colorful' == value or 'animal stickers' == value:
        return 'multicolor'
    if 'midnight' == value:
        return 'blue'
    return pd.NA

amazon_cleaned['color'] = amazon_cleaned['color'].apply(clean_color)
amazon_cleaned['color'].unique()


array([<NA>, 'white', 'black', 'grey', 'silver', 'multicolor', 'blue',
       'pink'], dtype=object)

In [11]:
# Adding columns based on 'connectivity technology' and setting values as per the given conditions

# Function to check connectivity and assign values
def check_connectivity(row, keyword, reject = []):
    if type(row)!=str:
        return -1
    if 'connectivity technology data not available' in row:
        return 0
    for r in reject:
        if r in row:
            return -1
    if any(kw in row for kw in keyword):
        return 1
    return -1

amazon_cleaned['wifi connectivity'] = amazon_cleaned['connectivity technology'].apply(lambda x: check_connectivity(x, ['wifi', 'wi-fi', 'wi fi']))
amazon_cleaned['ethernet connectivity'] = amazon_cleaned['connectivity technology'].apply(lambda x: check_connectivity(x, ['ethernet']))
amazon_cleaned['usb connectivity'] = amazon_cleaned['connectivity technology'].apply(lambda x: check_connectivity(x, ['usb']))
amazon_cleaned['t-tip connectivity'] = amazon_cleaned['connectivity technology'].apply(lambda x: check_connectivity(x, ['t-tip', 't tip', 'tip']))
amazon_cleaned['charger connectivity'] = amazon_cleaned['connectivity technology'].apply(lambda x: check_connectivity(x, ['charger', 'powerline']))
amazon_cleaned['magnetic connectivity'] = amazon_cleaned['connectivity technology'].apply(lambda x: check_connectivity(x, ['magnetic']))

amazon_cleaned[['connectivity technology', 'wifi connectivity', 'ethernet connectivity', 'usb connectivity', 't-tip connectivity', 'charger connectivity', 'magnetic connectivity']].head()


,connectivity technology,wifi connectivity,ethernet connectivity,usb connectivity,t-tip connectivity,charger connectivity,magnetic connectivity
0,connectivity technology data not available,0,0,0,0,0,0
1,usb,-1,-1,1,-1,-1,-1
2,connectivity technology data not available,0,0,0,0,0,0
3,connectivity technology data not available,0,0,0,0,0,0
4,"wi-fi, ethernet",1,1,-1,-1,-1,-1


In [12]:
amazon_cleaned['connector type'].unique()

array(['connector type data not available', 'usb type c',
       '4.5 x 3.0mm blue tip', 'magsafe', 'usb', 't tip',
       'barrel connector', 'round power connector', 't-type', '6-pin',
       '2-pin', 't-tip', '4.5'], dtype=object)

In [13]:
amazon_cleaned['barrel connector'] = amazon_cleaned['connector type'].apply(lambda x: check_connectivity(x, ['wifi', 'wi-fi', 'wi fi']))
amazon_cleaned['round power connector'] = amazon_cleaned['connector type'].apply(lambda x: check_connectivity(x, ['round power connector']))
amazon_cleaned['usb connector'] = amazon_cleaned['connector type'].apply(lambda x: check_connectivity(x, ['usb']))
amazon_cleaned['t-tip connector'] = amazon_cleaned['connector type'].apply(lambda x: check_connectivity(x, ['t-tip', 't tip', 'tip', 't-type']))
amazon_cleaned['type c connector'] = amazon_cleaned['connector type'].apply(lambda x: check_connectivity(x, ['type c']))
amazon_cleaned['magsafe connector'] = amazon_cleaned['connector type'].apply(lambda x: check_connectivity(x, ['magsafe']))
amazon_cleaned['6-pin connector'] = amazon_cleaned['connector type'].apply(lambda x: check_connectivity(x, ['6-pin']))
amazon_cleaned['2-pin connector'] = amazon_cleaned['connector type'].apply(lambda x: check_connectivity(x, ['2-pin']))

amazon_cleaned[['connector type', 'barrel connector', 'round power connector', 'usb connector', 't-tip connector', 
                'type c connector', 'magsafe connector', '6-pin connector', '2-pin connector']].head()

,connector type,barrel connector,round power connector,usb connector,t-tip connector,type c connector,magsafe connector,6-pin connector,2-pin connector
0,connector type data not available,-1,-1,-1,-1,-1,-1,-1,-1
1,usb type c,-1,-1,1,-1,1,-1,-1,-1
2,connector type data not available,-1,-1,-1,-1,-1,-1,-1,-1
3,4.5 x 3.0mm blue tip,-1,-1,-1,1,-1,-1,-1,-1
4,connector type data not available,-1,-1,-1,-1,-1,-1,-1,-1


In [14]:
amazon_cleaned['compatible devices'].unique()

array(['compatible devices data not available', nan, 'laptop',
       'smartphone, personal computer, security camera, smart television, tablet, printer, gaming console',
       '13"/15" /16" macbook pro macbook air 2021/ 2020/2019/2018/2017/2016',
       'tablets, laptops, cellular phones', 'dell computers',
       'usb flash drives, tablets, gaming consoles, laptops, desktops',
       'laptops', 'samsung', '65w lenovo laptop charger',
       'dell notebooks/ tablet pc', 'lenovo laptop charger 65w',
       'personal computer, router, ethernet connected devices',
       'mac pro, imac, ipad pro, chromebook, pixelbook, dell xps, samsung, and more pc, laptop, tablet',
       'pc, laptops', 'router',
       'laptop, gaming console, personal computer, tablet',
       'all wifi enabled devices', '65w asus laptop charger',
       'for hp pavilion charger, for hp charger for laptop, for hp computer charger, for hp laptop charger',
       'lenovo chromebook charger, lenovo laptop charger usb c

In [15]:
amazon_cleaned['compatible with tablet'] = amazon_cleaned['compatible devices'].apply(lambda x: check_connectivity(x, ['laptop', 'smartphone', 'macbook', 'desktop', 'notebook', 'mac', 'ipad', 'chromebook', 'pixelbook', 'thinkpad'], reject=['laptop charger', 'computer charger']))
amazon_cleaned['compatible with pc'] = amazon_cleaned['compatible devices'].apply(lambda x: check_connectivity(x, ['personal computer', 'pc', 'computer']))
amazon_cleaned['compatible with security camera'] = amazon_cleaned['compatible devices'].apply(lambda x: check_connectivity(x, ['security camera']))
amazon_cleaned['compatible with TV'] = amazon_cleaned['compatible devices'].apply(lambda x: check_connectivity(x, ['smart television', 'smart tv']))
amazon_cleaned['compatible with printer'] = amazon_cleaned['compatible devices'].apply(lambda x: check_connectivity(x, ['printer']))
amazon_cleaned['compatible with gaming console'] = amazon_cleaned['compatible devices'].apply(lambda x: check_connectivity(x, ['gaming console']))
amazon_cleaned['compatible with cellular phones'] = amazon_cleaned['compatible devices'].apply(lambda x: check_connectivity(x, ['cellular phone', 'cell phone']))
amazon_cleaned['compatible with usb'] = amazon_cleaned['compatible devices'].apply(lambda x: check_connectivity(x, ['usb']))
amazon_cleaned['compatible with flash'] = amazon_cleaned['compatible devices'].apply(lambda x: check_connectivity(x, ['flash']))
amazon_cleaned['compatible with router'] = amazon_cleaned['compatible devices'].apply(lambda x: check_connectivity(x, ['router', 'wifi']))
amazon_cleaned['compatible with projector'] = amazon_cleaned['compatible devices'].apply(lambda x: check_connectivity(x, ['projector']))

amazon_cleaned[['compatible devices', 'compatible with tablet', 'compatible with pc', 'compatible with security camera', 'compatible with TV',
                'compatible with printer', 'compatible with gaming console', 'compatible with cellular phones', 'compatible with usb', 
                'compatible with flash', 'compatible with router', 'compatible with projector']].head()

,compatible devices,compatible with tablet,compatible with pc,compatible with security camera,compatible with TV,compatible with printer,compatible with gaming console,compatible with cellular phones,compatible with usb,compatible with flash,compatible with router,compatible with projector
0,compatible devices data not available,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,compatible devices data not available,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,laptop,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [16]:
# Standardizing the date format in the 'date first available' column
from datetime import datetime

# Function to standardize date format
def standardize_date(date_str):
    if pd.isna(date_str) or 'date first available data not available' in date_str:
        return pd.NA
    try:
        return datetime.strptime(date_str.strip(), '%B %d, %Y').strftime('%Y-%m-%d')
    except ValueError:
        return pd.NA

amazon_cleaned['date first available'] = amazon_cleaned['date first available'].apply(standardize_date)
amazon_cleaned[['date first available']].head()

,date first available
0,<NA>
1,2022-05-17
2,<NA>
3,2020-06-08
4,2020-08-28


In [17]:
amazon_cleaned.to_csv('amazon_data_cleaned.csv')